import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
# Step 1: Import Libraries

# Step 2: Load Datasets
customers = pd.read_csv("Customers.csv")
products = pd.read_csv("Products.csv")
transactions = pd.read_csv("Transactions.csv")

# Step 3: Basic Overview of Each Dataset
# Customers Dataset
print("Customers Dataset Info:")
print(customers.info())
print("\nSample Data:\n", customers.head())
print("\nSummary Statistics:\n", customers.describe(include="all"))

# Products Dataset
print("\nProducts Dataset Info:")
print(products.info())
print("\nSample Data:\n", products.head())
print("\nSummary Statistics:\n", products.describe(include="all"))

# Transactions Dataset
print("\nTransactions Dataset Info:")
print(transactions.info())
print("\nSample Data:\n", transactions.head())
print("\nSummary Statistics:\n", transactions.describe(include="all"))

# Step 4: Check for Missing Values and Duplicates
print("\nMissing Values:")
print("Customers:\n", customers.isnull().sum())
print("Products:\n", products.isnull().sum())
print("Transactions:\n", transactions.isnull().sum())

print("\nDuplicate Records:")
print("Customers Duplicates:", customers.duplicated().sum())
print("Products Duplicates:", products.duplicated().sum())
print("Transactions Duplicates:", transactions.duplicated().sum())

# Step 5: Merge Datasets for Comprehensive EDA
# Merging on CustomerID and ProductID
merged_data = pd.merge(transactions, customers, on="CustomerID", how="inner")
merged_data = pd.merge(merged_data, products, on="ProductID", how="inner")

print("\nMerged Data Info:")
print(merged_data.info())
print("\nSample Merged Data:\n", merged_data.head())

# Drop Price_x and Price_y, create Price from Price_x
if 'Price_x' in merged_data.columns and 'Price_y' in merged_data.columns:
    merged_data['Price'] = merged_data['Price_x']
    merged_data = merged_data.drop(['Price_x', 'Price_y'], axis=1)


print("\nMerged Data Info:")
print(merged_data.info())
print("\nSample Merged Data:\n", merged_data.head())

# Extract product ID
merged_data['ProductID'] = merged_data['ProductID'].str.extract('(\d+)')
merged_data['TransactionID'] = merged_data['TransactionID'].str.extract('(\d+)')
merged_data['CustomerID'] = merged_data['CustomerID'].str.extract('(\d+)')

for col in ['TransactionID', 'CustomerID', 'ProductID']:
    try:
        merged_data[col] = pd.to_numeric(merged_data[col], errors='coerce')
    except KeyError:
        print(f"Column '{col}' not found in the DataFrame.")
    except Exception as e:
        print(f"An error occurred while converting '{col}': {e}")

Customers Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   CustomerID    200 non-null    object
 1   CustomerName  200 non-null    object
 2   Region        200 non-null    object
 3   SignupDate    200 non-null    object
dtypes: object(4)
memory usage: 6.4+ KB
None

Sample Data:
   CustomerID        CustomerName         Region  SignupDate
0      C0001    Lawrence Carroll  South America  2022-07-10
1      C0002      Elizabeth Lutz           Asia  2022-02-13
2      C0003      Michael Rivera  South America  2024-03-07
3      C0004  Kathleen Rodriguez  South America  2022-10-09
4      C0005         Laura Weber           Asia  2022-08-15

Summary Statistics:
        CustomerID      CustomerName         Region  SignupDate
count         200               200            200         200
unique        200               200              4  

In [ ]:
data = pd.merge(transactions, customers, on="CustomerID", how="inner")
data = pd.merge(data, products, on="ProductID", how="inner")
data['CustomerID'] = data['CustomerID'].str.extract('(\d+)')
data['CustomerID'] = data['CustomerID'].astype(int)

In [ ]:
customers['CustomerID'] = customers['CustomerID'].str.extract('(\d+)')
customers['CustomerID'] = customers['CustomerID'].astype(int)

In [ ]:
customer_profiles = data.groupby("CustomerID").agg(
    TotalSpend=("TotalValue", "sum"),
    AvgTransactionValue=("TotalValue", "mean"),
    TotalQuantity=("Quantity", "sum"),
    MostPurchasedCategory=("Category", lambda x: x.mode()[0]),
).reset_index()

# One-hot encode categorical variables (e.g., region and favorite category)
customers['RegionEncoded'] = pd.factorize(customers['Region'])[0]
customer_profiles["MostPurchasedCategory"] = pd.factorize(customer_profiles["MostPurchasedCategory"])[0]

customer_profiles = pd.merge(customer_profiles, customers[['CustomerID', 'RegionEncoded']], on='CustomerID', how='left')

# Step 5: Fill Missing Values
customer_profiles.fillna(0, inplace=True)

# Step 6: Normalize Numerical Features
scaler = MinMaxScaler()
numerical_columns = ["TotalSpend", "AvgTransactionValue", "TotalQuantity"]
customer_profiles[numerical_columns] = scaler.fit_transform(customer_profiles[numerical_columns])

if 'Region' in customer_profiles.columns:
    customer_profiles =customer_profiles.drop(columns=['Region'])
    print("Region column dropped successfully.")
else:
    print("Region column not found in the DataFrame.")

# Step 7: Compute Similarity
# Compute cosine similarity matrix for all customers
similarity_matrix = cosine_similarity(customer_profiles)
similarity_df = pd.DataFrame(similarity_matrix, index=customer_profiles.index, columns=customer_profiles.index)

# Step 8: Generate Recommendations
# Function to get top 3 similar customers for each customer
def get_top_3_similar(customers, similarity_df):
    lookalike_map = {}
    for customer_id in customers:
        similar_customers = similarity_df[customer_id].sort_values(ascending=False)
        similar_customers = similar_customers[similar_customers.index != customer_id]
        top_3_similar = similar_customers[:3]
        lookalike_map[customer_id] = list(zip(top_3_similar.index, top_3_similar.values))
    return lookalike_map

# Get top 3 lookalikes for the first 20 customers
customer_ids = customers["CustomerID"][:20]
lookalike_map = get_top_3_similar(customer_ids, similarity_df)

# Step 9: Save Recommendations
# Convert to DataFrame and save as Lookalike.csv
lookalike_list = []
for cust_id, lookalikes in lookalike_map.items():
    lookalike_list.append({"CustomerID": cust_id, "Lookalikes": lookalikes})

lookalike_df = pd.DataFrame(lookalike_list)
lookalike_df.to_csv("Lookalike.csv", index=False)

# Step 10: Display Lookalikes
print("Lookalike Recommendations for First 20 Customers:")
print(lookalike_df.head())

Region column not found in the DataFrame.
Lookalike Recommendations for First 20 Customers:
   CustomerID                                         Lookalikes
0           1  [(7, 0.957572650722631), (8, 0.928999528470089...
1           2  [(3, 0.9970135030408568), (5, 0.99036957102828...
2           3  [(2, 0.9970135030408568), (5, 0.97945277905464...
3           4  [(6, 0.9984401740783491), (15, 0.9934033649355...
4           5  [(2, 0.9903695710282813), (3, 0.97945277905464...
